In [1]:
import os

In [2]:
%pwd

'e:\\Aii_internship\\Ai_internshipp\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from keyword_extration.constants import *
from keyword_extration.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        # Create a DataIngestionConfig object based on the current setup
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
            # Remove the unzip_dir since we don't need it for JSON files
        )

        return data_ingestion_config


In [7]:
import os
import urllib.request as request
import zipfile
from keyword_extration import logger
from keyword_extration.utils.common import get_size

In [8]:
import os
import json
import logging
from pathlib import Path
import requests  # Assuming you might use requests for downloading
from typing import Any

logger = logging.getLogger(__name__)

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            response = requests.get(self.config.source_URL)
            with open(self.config.local_data_file, 'wb') as file:
                file.write(response.content)
            logger.info(f"{self.config.local_data_file} downloaded successfully!")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def load_json_file(self) -> Any:
        """
        Load JSON file from the specified path.
        Returns the content of the JSON file as a Python object.
        """
        with open(self.config.local_data_file, 'r') as json_file:
            data = json.load(json_file)
            logger.info(f"Loaded JSON file: {self.config.local_data_file}")
            return data


In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    
except Exception as e:
    raise e

[2024-10-26 11:44:13,394: INFO: common: YAML file: config\config.yaml loaded successfully]
[2024-10-26 11:44:13,396: INFO: common: YAML file: params.yaml loaded successfully]
[2024-10-26 11:44:13,400: INFO: common: YAML file: schema.yaml loaded successfully]
[2024-10-26 11:44:13,401: INFO: common: Created directory at: artifacts]
[2024-10-26 11:44:13,401: INFO: common: Created directory at: artifacts/data_ingestion]


TypeError: __init__() missing 1 required positional argument: 'unzip_dir'